In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from xgboost import XGBClassifier
import xgboost as xgb

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/jointsfull-data/full.csv
/kaggle/input/datmin-joints-2020/train_data.csv
/kaggle/input/datmin-joints-2020/test_data.csv
/kaggle/input/datmin-joints-2020/sample_submission.csv


In [13]:
full = pd.read_csv('/kaggle/input/jointsfull-data/full.csv')
y_ = pd.read_csv('/kaggle/input/datmin-joints-2020/train_data.csv').Result
submission = pd.read_csv('/kaggle/input/datmin-joints-2020/sample_submission.csv')

In [14]:
train = full.iloc[:len(y_)]
test = full.iloc[len(y_):]

## XGBoost From scratch

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train,y_,test_size=0.3, stratify=y_,random_state=42)
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label= y_valid)

In [16]:
train_d = xgb.DMatrix(train,label = y_)
test_d = xgb.DMatrix(test)

In [17]:
params = {'objective':'binary:logistic'}
num_rounds = 5

watchlist = [(dvalid,'valid'), (dtrain,'train')]
bst = xgb.train(params,dtrain,num_rounds,watchlist)

[0]	valid-error:0.20350	train-error:0.14404
[1]	valid-error:0.19153	train-error:0.11760
[2]	valid-error:0.17680	train-error:0.10261
[3]	valid-error:0.17588	train-error:0.08682
[4]	valid-error:0.16759	train-error:0.07459


In [18]:
preds_prob = bst.predict(test_d)
submission['Result'] = preds_prob
submission['Result'] = submission.Result.apply(lambda x: 1 if x > 0.5 else 0)
submission.head()

,id,Result
0,3621,0
1,3622,0
2,3623,0
3,3624,0
4,3625,0


## XGBClassifier

In [ ]:
params = {'objective':'binary:logistic'}
xgb = XGBClassifier(**params)
xgb.fit(train,y_)

In [ ]:
pred = xgb.predict(test)
submission['Result'] = preds_prob
submission['Result'] = submission.Result.apply(lambda x: 1 if x > 0.5 else 0)
submission.head()